# Part1 Data cleaning

## 1. Load Libraries and Raw Data (加载库和原始数据)

In this step, we import the essential libraries (Pandas, NumPy, and regex) and load the raw dataset. We quickly preview the first few rows to understand the raw structure.
这里导入所需的库并读取原始数据，同时展示前几行，快速了解数据结构。

In [2]:
import sys
!{sys.executable} -m pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np
import re

df = pd.read_excel("house_data.xlsx")
df.head()

,Unnamed: 0,info,loc,price
0,0,3室2厅 | 118.16平米 | 南 北 | 简装 | 高楼层(共28层) | 板楼,保利华都-虎泉杨家湾,378万
1,0,4室2厅 | 133.51平米 | 南 | 毛坯 | 低楼层(共32层) | 塔楼,东原启城一期-白沙洲,215万
2,0,3室2厅 | 93.09平米 | 南 | 精装 | 中楼层(共33层) | 板楼,保利心语九期-珞狮南路,168万
3,0,3室1厅 | 123.56平米 | 南 北 | 精装 | 中楼层(共34层) | 板楼,保利华都-虎泉杨家湾,355万
4,0,3室1厅 | 95.65平米 | 东南 | 精装 | 低楼层(共33层) | 板楼,美好长江首玺-白沙洲,118万


## 2. Data cleaning and Extraction (数据清洗和拆分)

2.1 

Split "info"

拆分“info”字段

In [11]:
#The original price values are stored in Chinese format, such as "378万". We remove the Chinese character “万” and multiply by 10,000 to convert the value into RMB (yuan)
#移除“万”，用数字替代
df["price"] = df["price"].str.replace("万", "", regex=False).astype(float) * 10000
# Extract the number of rooms and halls from the "info" column using regular expressions
#用正则表达式从“info”列中提取室和厅的数量
df["rooms"] = df["info"].str.extract(r"(\d+)室").astype(float)
df["halls"] = df["info"].str.extract(r"(\d+)厅").astype(float)
# Extract the area in square meters from the "info" column using regular expressions
#用正则表达式从“info”列中提取面积（平米）
df["area"] = df["info"].str.extract(r"(\d+\.?\d*)平米").astype(float)
# Extract the orientation 
#提取朝向
df["orientation"] = df["info"].str.extract(r"\|\s*([东南西北\s]+)\s*\|")[0]
df["orientation"] = df["orientation"].str.replace(" ", ",")
# Extract the decoration status
#提取装修情况
df["decoration"] = df["info"].str.extract(r"\|\s*(精装|简装|毛坯)\s*\|")
# Extract floor level and total floors
#提取楼层和总楼层
df["floor_level"] = df["info"].str.extract(r"(高|中|低)(?=楼层)")
df["total_floor"] = df["info"].str.extract(r"共(\d+)层").astype(float)
# Extract building type
#提取建筑类型
df["building_type"] = df["info"].str.extract(r"\|\s*(板楼|塔楼)")
# Delete the original "info" column as it's no longer needed
#删除原始的“info”列，因为不再需要
df.drop(columns=["info"], inplace=True)
df.head()

,Unnamed: 0,loc,price,rooms,halls,area,orientation,decoration,floor_level,total_floor,building_type
0,0,保利华都-虎泉杨家湾,3780000.0,3.0,2.0,118.16,"南,北,",简装,高,28.0,板楼
1,0,东原启城一期-白沙洲,2150000.0,4.0,2.0,133.51,"南,",毛坯,低,32.0,塔楼
2,0,保利心语九期-珞狮南路,1680000.0,3.0,2.0,93.09,"南,",精装,中,33.0,板楼
3,0,保利华都-虎泉杨家湾,3550000.0,3.0,1.0,123.56,"南,北,",精装,中,34.0,板楼
4,0,美好长江首玺-白沙洲,1180000.0,3.0,1.0,95.65,"东南,",精装,低,33.0,板楼


2.2

Split Location Column

拆分 loc 字段


In [12]:
#split the location field into “community name” and “subdistrict”.
#将“loc”字段拆分为小区名与片区名
df["community"] = df["loc"].str.split("-").str[0]
df["subdistrict"] = df["loc"].str.split("-").str[1]
df.drop(columns=["loc"], inplace=True)
df = df.drop(columns=["Unnamed: 0"])
df = df.dropna(subset=["area", "price"])

## 3.Preview and Save the Cleaned Dataset(预览和保存)


In [13]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3130 entries, 0 to 3129
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          3130 non-null   float64
 1   rooms          3130 non-null   float64
 2   halls          3130 non-null   float64
 3   area           3130 non-null   float64
 4   orientation    3130 non-null   object 
 5   decoration     2553 non-null   object 
 6   floor_level    3036 non-null   object 
 7   total_floor    3042 non-null   float64
 8   building_type  2491 non-null   object 
 9   community      3130 non-null   object 
 10  subdistrict    3130 non-null   object 
dtypes: float64(5), object(6)
memory usage: 269.1+ KB


,price,rooms,halls,area,orientation,decoration,floor_level,total_floor,building_type,community,subdistrict
0,3780000.0,3.0,2.0,118.16,"南,北,",简装,高,28.0,板楼,保利华都,虎泉杨家湾
1,2150000.0,4.0,2.0,133.51,"南,",毛坯,低,32.0,塔楼,东原启城一期,白沙洲
2,1680000.0,3.0,2.0,93.09,"南,",精装,中,33.0,板楼,保利心语九期,珞狮南路
3,3550000.0,3.0,1.0,123.56,"南,北,",精装,中,34.0,板楼,保利华都,虎泉杨家湾
4,1180000.0,3.0,1.0,95.65,"东南,",精装,低,33.0,板楼,美好长江首玺,白沙洲


In [14]:
df.to_csv("house_data_cleaned.csv", index=False)